# Query of the Event Descriptor Collection

This example illustartes the V4 interface for accessing the event_descriptor collection of the MongoDB-based Meta Data Store of the NSLS-II experimental facility

First, we import the necessary modules from the python standard library, numpy and eapy.

In [1]:
import sys
import pdb
import time
from datetime import timedelta, datetime, tzinfo

import numpy as np

#import the eapy module
sys.path.append('/home/malitsky/opt/eapy')
import eapy

and define parameters of the EPICS V4 RPC service, such as a channel name and timeout

In [2]:
chName = "ea4.rpc:mongo"; # service based on MongoDB
timeout = 30.0;

Then, we need to start a session for connecting it with the middle layer service and creating a session-specific context.

In [3]:
session = eapy.Session.createSession();
session.open(chName, timeout);

Now we can create a local proxy associated with the MongoDB collection

In [5]:
event_descriptorProxy = session.createProxy("event_descriptor"); 

and apply an action to collect data sets

In [6]:
t1 = datetime.now();
event_descriptorCollect = event_descriptorProxy.collect(timeout);
t2 = datetime.now()
print "processing time: ", (t2 - t1);

processing time:  0:00:00.708231


The collect method returns a structure including a query status and a collection of documents. First, let's check the query status (int type, string message, string callTree). There are four types of query statuses: 0 - OK, 1 - Warning, 2 - Error, 3 - Fatal. 

In [7]:
print "status:", event_descriptorCollect['status']['type']

status: 0


If the query is OK, we can extract an array of documents.

In [8]:
docs = event_descriptorCollect['docs']
print "size:", docs['size']

size: 1941


Also we can select a few entries by combining filter() and collect()

In [10]:
event_descriptorFilter = event_descriptorProxy.filter([
{"$match" : {"uid" : {"$regex" : "197d5a64-3748-40b9-a257-6b89208b8010"}}}]);
event_descriptorCollect2 = event_descriptorFilter.collect(timeout) 
print event_descriptorCollect2['status']['type']

0


and get corresponding documents:

In [11]:
docs = event_descriptorCollect2['docs']
print "size:", docs['size']

size: 1


According to the schema, the event descriptor has a composite structure consisting of common properties, like run_start_id, uid, time

In [16]:
doc = docs['0']
print doc['run_start_id'], doc['uid'], datetime.fromtimestamp(int(doc['time'])), len(doc['data_keys'].keys())

54e3c77c7368e3237a36b289 197d5a64-3748-40b9-a257-6b89208b8010 2015-02-17 17:58:23 7


and a descriptor-specific collection of data keys

In [15]:
data_keys = doc['data_keys']
for key in data_keys.keys():
    data_key = data_keys[key]
    print key 
    if isinstance(data_key, eapy.Document) :
        anames = data_key.keys()
        atypes = data_key.types()
        for i in range(len(anames)):
            if(atypes[i] == "numpy.ndarray") :
                print "    ", anames[i], data_key.ndarray(anames[i])
            else :
                print "    ", anames[i], data_key[anames[i]]

theta
     dtype number
     shape []
     source XF:23ID1-ES{Dif-Ax:Th}Mtr.RBV
sclr_ch2
     dtype number
     shape []
     source PV:XF:23ID1-ES{Sclr:1}.S2
sclr_ch3
     dtype number
     shape []
     source PV:XF:23ID1-ES{Sclr:1}.S3
sclr_ch1
     dtype number
     shape []
     source PV:XF:23ID1-ES{Sclr:1}.S1
sclr_ch6
     dtype number
     shape []
     source PV:XF:23ID1-ES{Sclr:1}.S6
sclr_ch4
     dtype number
     shape []
     source PV:XF:23ID1-ES{Sclr:1}.S4
sclr_ch5
     dtype number
     shape []
     source PV:XF:23ID1-ES{Sclr:1}.S5
